In [ ]:
#@title Data-AI（必ず自分の名前・学籍番号を入力すること） { run: "auto", display-mode: "form" }

import urllib.request as ur
import urllib.parse as up
Name = '\u591A\u7530 \u5DEB\u5973\u7F8E' #@param {type:"string"}
EName = 'Tata Mikomi' #@param {type:"string"}
StudentID = '87654321' #@param {type:"string"}
Addrp = !cat /sys/class/net/eth0/address
Addr = Addrp[0]
url = 'https://class.west.sd.keio.ac.jp/classroll.php'
params = {'class':'dataai','name':Name,'ename':EName,'id':StudentID,'addr':Addr,
           'page':'dataai-text-9','token':'27652099'}
data = up.urlencode(params).encode('utf-8')
#headers = {'itmes','application/x-www-form-urlencoded'}
req = ur.Request(url, data=data)
res = ur.urlopen(req)

---
> 畳み込むの意味は、折りたたんで中に入れること、心の中に深く留めること、工学的には移動しながら共有部分の和を求めていくこと
> 
> 畳み込み演算に意味ができることの本質は線形で時不変であること、受けた影響の総和が求まるということ
>
> RC回路に方形波を入力する時その出力を、RC回路のインパルス応答曲線と、方形波との積分を、方形波を移動させながら求めて応答を得るのが畳み込み積分
>
> 源画像データとフィルタ行列とのアダマール積を移動しながら求めるのが畳み込み層
>
> この授業も教師曲線と学生曲線の畳み込み値が大きくなるようにしたい
>
> それ以前に線形で時不変であることが難しいのだが
---

# 畳み込み層とプーリング層

## 畳み込み層について

畳み込みニューラルネットワーク、Convolutional Neural Network (CNN)については既に説明済みである

特に画像に用いるが、画像など2次元データから特徴を効率よく集めることができる仕組みといえる

CNNについての本質的な理解は、マルチメディアデザインの授業を受けた学生は画像フィルタリングの動作原理を理解していれば掴みやすいであろう
- 画像とオペレータのアダマール積により、エッジ抽出や線分抽出などの画像フィルタが記述できる
- オペレータを工夫すれば、他にも様々な特徴が抽出できることはマルチメディアデザインの授業でも紹介した通りである

なお、後で確認できるが、画像に対して全結合網で次元削減しながら学習させるのと、CNNを使うのでは明確にCNNの方が性能がよくなる

なお、次の図のように複数のフィルタを同時に用いることが多い
- この図において情報の次元数であるチャネル数の移り変わりについてまとめる
- 画像の画素の位置(x,y)の2次元の情報のチャネルに加えてRGBの色情報チャネルを含む3チャネルあるが、同数の3枚のフィルタが必要となる
- フィルタと畳み込み演算を行ったRGB各チャネルのデータを合計して1チャネルのデータとする
- バイアスを与えて活性化関数による演算を施す
- 結果としてフィルタ数分のチャネルの生成データが得られる
  - 画像としてみることもできるが、画像と呼べるかどうかは微妙であるため生成データとここでは表現している

<img src="http://class.west.sd.keio.ac.jp/dataai/text/cnnch.png" width="60%">

畳み込み層の処理は、2次元だけでなく、1次元や3次元にも拡張でき、PyTorchはネイティブでconv1d, conv2d, conv3dを備える
- conv4dなどもGitHubなどで利用できる

オプションは次の通り

| | |
|:--|:--|
|in_channels (int)| 入力チャネル数、最初の層なら通常RGBで3 |
|out_channels (int) | 出力チャネル数でフィルタ数 |
|kernel_size (int or tuple) |各フィルタカーネルのサイズ|
|stride (int=1 or tuple, optional)|ストライド|
|padding (int=0 or tuple, optional)|パディング幅|
|padding_mode (string='zero', optional)|多次元テンソルに対するパディング層の考慮に関する内容で指定<br>することはないであろう<br>なおpadというパディング専用の柔軟な関数も準備されている|
|dilation (int=1 or tuple, optional)|計算したカーネルの値を並べる際のストライド|
|groups (int=1, optional)| 入力から出力へのカーネルの分割ブロック数で並列化の指定<br>一般にgroup convolutionとは異なる複数のカーネルを一つの入力に<br>適用することであるが、その文脈では定義されていない|
|bias (bool=1, optional)|True=1であれば、学習可能なバイアスを追加する|


## プーリング層について

こちらも既に説明済みであるが、ある領域の最大や平均といった値を取得してさらにデータサイズを縮約する
- 一般的には最大プーリングを利用する
- 平均プーリングは2016年に提案された画像物体認識DNNであるLeNetで利用されたが、普通利用しない
- ややこしいが、ResNetでは全平均を計算するglobal average poolingが用いられる 

画像では一般的にmax poolingが用いられ、最も特徴的な値のみを次に伝えるという効果が得られる

pooling層なしにCNNを重ねてもよいのではないか？という考えもあると思うが、計算コストを削減する意味でCNNとpoolingを交互に重ねるような構成が多くみられる
- どのみち特徴を抽出するという明確な目的においては、特徴がある、つまり値の大きいところを選び出せばよいと考えられる
- poolingの方が演算コストが小さいことから、抽出するという処理であればpoolingを用いることで学習に必要な計算コストを削減できる





## ストライドについて

これも既に説明済みであるが、一般に畳み込み層ではストライドは1となり画像サイズに大きな差が出ない

プーリングではプーリング用のフィルタサイズに合わせてストライドが設定される
- つまり、プーリングの各演算範囲が重ならないようにずらす
- 最大値が欲しいのであるから、重なってはいけない
  - 重なると同じ最大値が複数回結果に表れてしまい、特徴が重複・強調される


## 出力データのサイズについて

次のように変数を定める
- 入力画像高さ,幅: $I_h$, $I_w$
- フィルタ高さ,幅: $F_h$, $F_w$
- パディング幅: $D$
- ストライド幅: $S$

すると、出力画像の高さ$O_h$と幅$O_w$は次のように表される
$$
O_h = \frac{I_h-F_h+2D}{S}+1\\
O_w = \frac{I_w-F_w+2D}{S}+1
$$

例:畳み込み層
- 入力画像: 32$\times$32、RGB 3チャネル
- フィルタ: 5$\times$5, フィルタ数6
- パディング: なし
- ストライド: 1

このとき、出力データは、
$(32-5+2\times 0)/1+1=28$となる

すなわち、6チャネルの28$\times$28のデータとなる

例:プーリング層
上記データを入力とする
- 入力画像: 28$\times$28、6チャネル
- 領域サイズ: 2$\times$2

このとき、出力データは、
$20/2=14$となる

すなわち、6チャネルの14$\times$14のデータが得られる

全体をまとめると次の通りとなる

| | | | |
|:-:|:-:|:-:|:-:|
| |**CONV**|**POOL**|**FC**|
|図|<img src="http://class.west.sd.keio.ac.jp/dataai/text/table-conv.png" width="150">|<img src="http://class.west.sd.keio.ac.jp/dataai/text/table-pool.png" width="150">|<img src="http://class.west.sd.keio.ac.jp/dataai/text/table-fc.png" width="150">|
|入力サイズ|$I \times I \times C$|$I \times I \times C$|$N_{\text{in}}$|
|出力サイズ|$O \times O \times K$|$O \times O \times C$|$N_{\text{out}}$|
|パラメータの数|$(F \times F \times C + 1) \cdot K$|$0$|$(N_{\text{in}} + 1 ) \times N_{\text{out}}$|
|備考|•フィルタ毎にバイアスがある<br>•一般に$S<F$<br>•一般に$K=2C$|•プール操作はチャネル毎<br>•一般に$S+F$|•ニューロン毎にバイアス1つ<br>•ニューロン数に構造的制約なし|


## 畳み込み層でよく利用される活性化関数

ReLUが一般的に用いられるが、次のような活性化関数が利用される
- 悩むぐらいならReLUで十分

| | | |
|:-:|:-:|:-:|
|ReLU|Leaky ReLU|ELU|
|$g(z)=\max(0,z)$|$g(z)=\max(\epsilon z,z)$<br>ただし$\epsilon\ll1$|$g(z)=\max(\alpha(e^z-1),z)$<br>ただし$\alpha\ll1$|
|<img src="http://class.west.sd.keio.ac.jp/dataai/text/figrelu.png" width="150">|<img src="http://class.west.sd.keio.ac.jp/dataai/text/figleaky-relu.png" width="150">|<img src="http://class.west.sd.keio.ac.jp/dataai/text/figelu.png" width="150">|
|•生物学的に解釈可能な構成かつ非線形|•ReLUの負値不感問題に対処|•どこでも微分可能|


# データ拡張(data augmentation)による汎化性能向上

データ拡張とは、汎化性能を向上するためにデータを水増しすることである

- 例えば、訓練データのサンプル数が少ない場合などにおいては、過学習が進みやすく汎化性を失う

- その単純な対策の一つが、**データを増やす**ということ、これがデータ拡張の意味である

## 改めて過学習とはどういう状態か

既に何度か扱っているので、ここでは一般的な説明とするが、欲しいモデルとは異なる、あるデータだけに特化したモデルが構築された状態のこと

学習曲線、検証曲線、ロス曲線において、学習データのロスとテストデータのロスの間が開いている状態を過学習状態と呼ぶ

- ただし、学習があまり進んでいない状態で開いている場合は、モデル構築が不十分で、未学習(アンダーフィッティング)というべき状態であり、過学習(オーバーフィッティング)とは言わない

- 学習がある程度進んだ状態で呼ぶ

- 学習が比較的早く進むadamを用いて比較的簡単なモデルを多めのエポック学習させてみるとよい

重要なので、改めてまとめる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/losscurve.png" width="60%">


## 実際のデータ拡張

画像データでは次のような方針がとられる
- 画像を様々な方向に回転させる
- 画像を拡大・縮小する
- 画像の平行移動
- 画像の上下・左右の反転
- 画像の一部を削除する(隠す)

# ドロップアウト(dropout)

具体的に何をするかは説明済みであるので、その用い方のテクニックについて述べる

- 出力層以外の通常ノードを一定確率でランダムに消去する
- 実装が容易で過学習を抑制でき、またその効果が大きい
- バッチごとに消去する結合を変更する

また、

- ドロップアウト率$p$に基づきノード単位でドロップアウトするかどうかを決定する

- CNNでは用いてはいけない

  - 重要な特徴を捕まえられなくなる
  - そもそもCNNはものすごい勢いで情報圧縮・次元圧縮している

- 解釈としては、ランダムフォレストで利用したアンサンブル学習と同じ効果が期待できる

  - Dropoutによりスリム化されたネットワークを組み合わせると解釈できる

Dropoutしても安定して学習できるようになってしまった、そういう技術ができてしまったということ

近年では、画像処理応用が旺盛で、CNNではdropboutを用いないため、あまり利用されていない

- Dropoutさせた方が学習で最適解に至る速度は低下する

- ならば、ちゃんと監視して過学習に至る前に学習をやめればよいという意見もある

# バッチノーマライゼーション(Batch Normalization)

Batch Normalization(以下BN)はシンプルでありながら、

> 特にGAN等安定化しにくいネットワークが利用可能になったのは、Batch NormalizationとAdamによる功績が大きく、なければノイズしか出てこない

といわれており、また、今後VAEについても学ぶが、

> VAEでも、うまくいかなかった事例が、Batch Normalizationを入れるといきなり学習が成功した

という事例も報告されている

BNはもともと、勾配消失・爆発を防ぐための手法として提案され、そのための対策として、
- 活性化関数をReLUなどに変更
- ネットワークの重みの初期値を事前学習しておく
- 学習係数を下げる(遅くなる)
- ネットワークの自由度を下げる
  - Dropoutもこの手法の一つ

といった対応が取られてきたが、これに対する新たな対応手法である

### バッチノーマライゼーションの処理内容

BNとは、各ユニットの出力をminibatchごとにnormalizeした新たな値で置き直すことで、内部の変数の分布(内部共変量シフト)が大きく変わるのを防ぎ、学習を進め、過学習を抑える手法である
- 要するに、ネットワークの途中で伝播している値を無理やり正規化する手法
- 特に**白色化(whitening)**と呼ばれる手法では、平均0、標準偏差1に途中変換すればニューラルネットワークの収束が速くなることが知られている
- ネットワークの層の中に$\boldsymbol{H}$という行列を配置する
  - 各行がminibatchの1つのデータ、各列がそれぞれのactivationとなるような値をとる行列
  - 例えばbatch sizeが128, hidden unitの数を256とすると、  $\boldsymbol{H}$ は128×256の行列

正規化のため、次のように$\boldsymbol{H}'$ と置き直すのがbatch normalizationの仕組み

$$
\boldsymbol{H}' = \frac{\boldsymbol{H}-\boldsymbol{\mu}}{\boldsymbol{\sigma}}, 
$$

- $\boldsymbol{\mu}$と$\boldsymbol{\sigma}$ は、その層での各ユニットの平均および標準偏差のベクトル

訓練時の$\boldsymbol{\mu}$と$\boldsymbol{\sigma}$ は次の式を用いて算出する

$$
\boldsymbol{\mu} = \frac{1}{m} \sum_i \boldsymbol{H}_{i,:}, \\
\boldsymbol{\sigma} = \sqrt{\delta + \frac{1}{m} \sum_{i}(\boldsymbol{H}-\boldsymbol{\mu})^2_i}
$$

- $\delta$ は標準偏差が0にならないようにする十分小さな値

推定時は、minibatchはないため、訓練時に利用した値の移動平均などを$\boldsymbol{\mu}$と$\boldsymbol{\sigma}$に適用する

### バッチノーマライゼーションの種類

処理方法は同一であるが、1度に正規化する範囲の違いから次のような種類がある
- バッチノーマライゼーション：ミニバッチの同一箇所(BachNormalization演算が割り当てられている同じ層)の間で正規化する
- レイヤーノーマライゼーション：異なるレイヤ(チャネル)全体で正規化する手法で、RNNやTransformerで用いられる
- グループノーマライゼーション：異なるいくつかのレイヤ(チャネル)で正規化する手法で、画像認識で用いられることがある
- インスタンスノーマライゼーション：1つのレイヤチャネルだけで正規化する手法で、StyleGANなどのGANで用いられる

### なぜバッチノーマライゼーションが有効なのか

まず、誤ってはいますが、現論文に記載されており、皆さんが信じた理由が、内部共変量シフト(= ICS, Internal Covariance Shift)の解消です
- ニューラルネットの層を過ぎる毎にその層のアウトプットの分布が変わる(これをICSと呼ぶ)が、このようにニューラルネットの中を通るデータの分布が頻繁に変化すると学習が困難になるというのは感覚的に理解できるであろう
- であれば、バッチノームはこのICSを正規化により解決し、精度向上に寄与すると考えられる

しかしながら、実際には調べたところBNではICSは解決されないため、この主張は誤っているであろうというのが現在の見解です
- ですが、この考え方には大きなヒントがあります
- ある重みを更新する際に、本来その前の各層の重みがどのように更新されるのかを考慮する必要がある
- BNを用いることで、その前の全体の層の関係を表現た値を利用できる
というのが、現在の解釈となる

### BN利用のメリット
- 学習係数を大きくすることができる
  - 学習係数を増加させるとパラメタの大きさが飛躍的に増え、勾配消失や勾配爆発につながるため大きくできなかった
  - 途中で正規化すればパラメタが飛躍的に大きくなることを防ぐことができる

- 正則化効果がある
  - L2正則化の必要性がほぼなくなる
  - Dropoutの必要性がほぼなくなる
    - 近年ではDropoutがあまり利用されていないが、これはBNの影響によるところが大きい
    - Dropoutは学習速度が低下するが、BNは逆に向上するので利用価値が薄れてしまった
- ネットワークの重みの初期値の影響を受けにくい

性能で悩んだら、まずはBNの利用を検討するべきである

PyTorchでは、画像の場合`BatchNorm2d`が提供されている

# MNISTのCNNによる分類

今まで何度も取り扱ってきたMNISTを改めてCNNを用いて評価する

プログラムも理解が進んでいると思うので、より行儀のよいスタイルにする

GPU対応の確認を行う
- PyTorchはGPUモードを明示的に指定する

In [ ]:
cuda = "cuda:0"
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
# ハイパーパラメータ 
num_epochs = 10
batch_size = 100
learning_rate = 0.001
# GPU存在のチェック
device = torch.device(cuda if torch.cuda.is_available() else "cpu")
print(device)

MNISTを読み込むDataSetとDataLoaderを作成する

In [ ]:
# MNISTデータセット
train_dataset = dsets.MNIST(root='mydata', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='mydata', 
                           train=False, 
                           transform=transforms.ToTensor())
# DatasetLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## モデルの作成

- CNNはブロック単位で処理した方がよいのでブロック（Conv+BN+ReLU+Pooling）ごとにまとめて Sequential を使うとわかりやすくなる
- Conv2d や BatchNorm2d は出力のユニットサイズをきちんと指定する
- サイズを自分で計算するのが面倒ならば、モデルの途中結果サイズを`print(out.size())`や`print(out.shape)`として出力して確認するとよい
  - ここではそのようにして調べた
- ネットワークの指定は画像1枚に対して記述しているように見えるが、実際にはバッチ数分並列して動作するため、推定結果もバッチ数分出力されることに注意する
  - つまり、画像一枚に対するノード数が、さらにバッチ数分一挙に出力される

このサイズの求め方は面倒なので、printしてみて納得したら、`out.size(0)`のように指定すると若干？お手軽である
- この場合`print(out.shape)`では`torch.Size([100, 32, 7, 7])`と表示される
- 先頭の100はバッチサイズで、これが`out.size(0)`である
  - ネットワークは内部では100個並列で作成され、推定結果も100個並列して出てくる
  - それが全て配列になって保存される、一つの画像当たり$32\times 7\times 7$の数のノードの出力が得られる
- 最終的には100個の並列した推定結果が欲しいので、`torch.Size([100,1568])`となってほしい
  - そこで、ノードの個数がわかるのならば、バッチサイズを未確定として`view(-1,1568)`としてもよいし、バッチサイズがわかるのであればノード数を未確定として`view(100,-1)`としてもよい
  - バッチサイズは、`torch.Size([100, 32, 7, 7])`なので、`out.size(0)`で得ることができ、`view(out.size(0), -1)`が成立する

後半のCIFAR-10では畳み込み層の出力サイズを求める計算式を示して計算してサイズを得るため、こちらのノードサイズを未確定とする例と違いバッチサイズを未確定としている
- どちらも同じであるが、`out.size(0)`を用いる方がお手軽である
- とはいえ、結局ネットワークの定義で、`nn.Linear(7 * 7 * 32, 10)`とする必要があるため、ノード数の情報は必要となることから、お手軽さは限定的

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(1, 16, kernel_size=5, padding=2),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(2))
    self.layer2 = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size=5, padding=2),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(2))
    self.fc = nn.Linear(7 * 7 * 32, 10)
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out
model = CNN().to(device)
model

In [ ]:
# テスト
images, labels = iter(train_loader).next()
print(images.size())

今回のモデルのように、Batch NormalizationやDropoutなど学習時と推論時で挙動が変わるレイヤを使う場合はモデルのモードに注意して、きちんと切り替えること
- model.train() で訓練モード
- model.eval() で評価モード
  - 切り替えなくてもエラーにはならないがそもそも結果に影響するなどの弊害が発生する
- torch.no_grad()も忘れずに
  - こちらは忘れてもよいが、特にGPUでは処理速度やメモリ効率の影響が大きくなる
また、GPUモードで動かすときはテンソルを下のように to(device) でGPUに送る必要がある

モデルオブジェクトを生成する

GPUモードで動かすには

- モデルを to(device) でGPUに転送する
- テンソルデータも to(device) でGPUに転送する

の2つだけ実装すればよい

In [ ]:
images = images.to(device)
labels = labels.to(device)

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 訓練

今後テンプレ的に使えるよう、ここでは精度も計算するが、精度を求めるのは若干工夫がいる
- **一気にまとめて調べ上げる方法**で普通によく見る記述
- まずモデルの出力が全検証のそれぞれのテンソルを持つ
- そこで、各テンソルの最大値を集めれば、それが予測ラベル
  - `predicted = outputs.max(1, keepdim=True)[1]`
- view_asはtensorとtensorを比較する時などで形を揃えたい時に便利な関数で、型をsize()で調べてviewで変換するなどとしなくても変換できる
  - viewは、直接、型を指定するが、view_asは他のテンソルの型に倣って変換してくれる
    - `labels.view_as(predicted)`でlabelテンソルをpredictedテンソルの形に変換
    - eqで複数を同時に比較、それぞれでTrueかFalseを得る
    - そのうちのTrueの数をsumで数える
    - Trueは1で、Falseは0扱いなので、sumするとTrueを数えるのと同じ
    - item()でPythonの普通の数字として取り出す\
    これを忘れると、`correct/total`の計算がtensorの割り算となりうまくいかない
    - これをcorrectに加える

- 別の方法として、**一つの結果であれば**`correct += (outputs.argmax(1) == labels).sum().item()`とすることもできる
  - 中身は同じだがargmaxで最大値となる**場所**を取得してそれがラベルと一致すれば正解、これを合計して、pythonの数字として取り出している
  - ただし**一つの結果だけ**であり、**全結果まとめて計算しているわけではない**ことに注意する

In [ ]:
def train(train_loader):
  model.train()
  running_loss = 0
  for batch_idx, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    loss.backward()
    optimizer.step()
  train_loss = running_loss / len(train_loader) 
  return train_loss

def valid(test_loader):
  model.eval()
  running_loss = 0
  correct = 0
  total = 0
  with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(test_loader):
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      running_loss += loss.item()
      predicted = outputs.max(1, keepdim=True)[1]
      correct += predicted.eq(labels.view_as(predicted)).sum().item() # ここで精度を計算
      total += labels.size(0)
  val_loss = running_loss / len(test_loader)
  val_acc = correct / total
  return val_loss, val_acc

loss_list = []
val_loss_list = []
val_acc_list = []
for epoch in range(num_epochs):
  loss = train(train_loader)
  val_loss, val_acc = valid(test_loader)
  print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f' % (epoch, loss, val_loss, val_acc))
  # logging
  loss_list.append(loss)
  val_loss_list.append(val_loss)
  val_acc_list.append(val_acc)

# save the trained model
np.save('loss_list.npy', np.array(loss_list))
np.save('val_loss_list.npy', np.array(val_loss_list))
np.save('val_acc_list.npy', np.array(val_acc_list))
torch.save(model.state_dict(), 'cnn.pkl')

## 結果表示

前回、多層パーセプトロンでの精度が85%くらいだったので99%程度出るCNNはかなり性能がよいことがわかる

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# plot learning curve
plt.figure()
plt.plot(range(num_epochs), loss_list, 'r-', label='train_loss')
plt.plot(range(num_epochs), val_loss_list, 'b-', label='test_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()

plt.figure()
plt.plot(range(num_epochs), val_acc_list, 'g-', label='val_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.grid()

# CIFAR-10のCNNによる分類


次は同じことをCIFAR-10で行う

初めに必要なライブラリをimportしておく

必要な時にそれぞれ読み込むのが本来は正しいのかもしれないが、テンプレート化した方が使い勝手が良い

なお、`from rotch.utils.data import DataLoader`としていたためDataLoaderで読み込めるが、ここでは、敢えてクラスを辿ってDataLoaderを指定する

同様に、`from torchvision.datasets import CIFAR10`とすると`train_set = CIFER10...`とできる

どちらでも好みでよい

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
device = torch.device(cuda if torch.cuda.is_available() else "cpu")
print(device)

ハイパーパラメータを宣言

In [ ]:
num_epochs = 20
batch_size = 128 # GPUを使うなら大きくてもよい
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

## CIFAR10データの確認

本来は不要だが、CIFAR10データの中身を確認してみる

- CIFAR-10はラベル付きの6万枚のカラー画像により構成されるデータセットで、画像サイズは$32 \times 32$
- ラベルも少なく、10種類のみ

普通にmathplotlibの機能を利用すると、次のような表示となる

In [ ]:
cifar10 = torchvision.datasets.CIFAR10(root="mydata",
                       train=False,download=True,
                       transform=transforms.ToTensor())
plt.figure(figsize=(7,7))  # 画像の表示サイズ
chkdata = iter(torch.utils.data.DataLoader(cifar10, batch_size=4*4, shuffle=False))
chkimages, chklabels = chkdata.next()  # イテレータから最初のバッチを取り出す
for i in range(4*4):
  plt.subplot(4,4,i+1)
  plt.imshow(np.transpose(chkimages[i], (1, 2, 0)))  # チャンネルを一番後ろへ
  label = classes[chklabels[i]]
  plt.title(label)
  plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # ラベルとメモリを非表示に
plt.show()

## データ拡張

### データの回転・反転・拡大縮小

CIFAR-10の画像情報をランダムに回転、左右反転および拡大縮小する

transforms.Composeにより配列で処理順序を指定する、つまり**順番が重要となる**

- 例えば、`Normalize`と`RandomAffine`の順番を逆にするとエラーになる
  - `RandomAffine()`はtorch型ではなく、PIL.Imagesを返すため、`ToTensor`よりも,前に`RandomAffine`を施す必要がある

  - これらの変換は**データを取得する度に実行される**ので、同じデータを取り出しても異なる回転、反転、拡大縮小の画像が得られることに注意する
  - appendして増やそうなどと考えなくともよい

### PyTorchが備えるデータ拡張手法

PyTorchには、データ拡張用に次の手法が準備されている
- RandomHorizontalFlip:  画像を確率pで左右反転する
- RandomAffine: 画像をランダムに回転/拡大縮小する
- RandomErasing: 画像をランダムな一部分にノイズを付加する
- RandomPerspective: 画像を確率pでランダムに透視変換、すなわち視角を変える

などがある、他にも便利な、
- Resize: 画像のリサイズ
- CenterCrop: 画像の中心に移動する

などの機能も備えている

### データの標準化

データ拡張とは関係ないが、CIFAR-10の画素情報をオリジナルの [0, 1] から[-1, 1] になるように標準化する

- `transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))`について、一つ目の引数のタプルはRGBの各チャンネルの平均を表し，二つ目の引数のタプルは標準偏差を表す
  - つまり0から1のデータの平均値はおそらく0.5に近い値であると思われるので、平均を$mean=0.5$として、これを正規化後0にする
  - $z=\frac{x-mean}{std}$として求められ、$std=0.5$とすれば、丁度取りうる値の範囲が1から2倍の2となる
   - これを全チャネルに対して処理する

- 画像を表示させてから標準化させてもよいが、途中結果を表示する際に変な絵が出てきてびっくりということになりかねない

  - ここでは、あえて先に標準化し、標準化した画像を表示させるようにしている

### DataLoader

num_workers を指定するとファイルの読み込みが並列化され、例えばCPUのコアが複数ある場合は一気に速くなる

`batch_size`は、`train_loader`では`batch_size`を指定しているが、`test_loader`では`len(test_set)`として全て食べさせている

In [ ]:
transform = transforms.Compose([
  transforms.RandomAffine([0,30], scale=(0.8, 1.2)), # 回転および拡大縮小
  transforms.RandomHorizontalFlip(p=0.5), # 左右反転
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # [0, 1] => [-1, 1]
])
train_set = torchvision.datasets.CIFAR10(root='mydata', train=True,
                          download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                          shuffle=True, num_workers=4)
test_set = torchvision.datasets.CIFAR10(root='mydata', train=False,
                          download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set),
                          shuffle=False, num_workers=4)
print("訓練データの数:", len(train_set))
print("検証データの数", len(test_set))

torchvisionには、次のように複数の絵を纏めて一つの絵にする機能がある

In [ ]:
def imshow(img):
  # unnormalize [-1, 1] => [0, 1]
  img = img / 2 + 0.5
  img = img.numpy()
  # [c, h, w] => [h, w, c]
  plt.imshow(np.transpose(img, (1, 2, 0)))
images, labels = iter(test_loader).next()
images, labels = images[:16], labels[:16]
imshow(torchvision.utils.make_grid(images, nrow=4, padding=1))
plt.axis('off')

PyTorchのチュートリアルに従い、ドロップアウト層を追加して別途CNNの簡単なモデルを定義する

レイヤの内容をよく確認すること
- 2つめの畳み込み層もプーリング層を伴っていることを確認する
  - forwardでそのように定義されている
- 次の全結合層はそれなりにノード数を大きくとっている
  - ドロップアウト層では、なんと50%ものノードの計算をスキップする
  - この全結合層の数を増やすのも面白いであろう
    - 各自試してみると良い
- 1つめの畳み込み層(conv1)の出力サイズ
  - 画像サイズは32$\times$32、フィルタサイズが5、パディングが0、ストライドが1なので、32-5+1=28
    - フィルタサイズはカーネルサイズとも呼ばれる
- 続くプーリングの出力サイズ
  - 2$\times$2なので、画像サイズは28$\div$2=14
- 続く畳み込み層(conv2)の出力サイズ
  - データサイズ14、フィルタサイズ5なので14-5+1=10
- 続く1次元化(view)では、フィルタ数16で各フィルタのサイズが10であるため、$16\times 5\times 5$の要素となる
- MNISTの例の通り、結局ミニバッチ数がわかっているので、`x.view(-1, 16 * 5 * 5)`という記述は、`x.view(x.size(0), -1)`でもよいが、いずれにせよノード数の情報は必要でそれほど得はしない

シンプルな画像認識について一番最初にデモを示したが、それでも23層、その何倍も層数の多いモデルも普通に存在する
- もうこうなると、どういうトポロジがよいのか確認するのも大変
- 経験がものをいう困った世界
  - AIなんて経験なんだよという矛盾

In [ ]:
class DNN(nn.Module):
  def __init__(self):
    super(DNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, kernel_size=5) # 畳み込み層(入力ch,フィルタ,そのサイズ)
    self.pool = nn.MaxPool2d(2, 2) # プーリング層(領域サイズ,ストライド)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5) # 2つめの畳み込み層
    self.fc1 = nn.Linear(16 * 5 * 5, 256) # 全結合層
    self.dropout = nn.Dropout(p=0.5) # ドロップアウト(ドロップアウト率)
    self.fc2 = nn.Linear(256, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
    return x

ReLUは nn.ReLU() で層として定義する場合もあるが、パラメータがないので F.relu() のように関数として使うこともできる

モデルを確認する

In [ ]:
model = DNN().to(device)
model

## 学習

モデルを訓練する

- DataLoaderを用いてミニバッチ毎に訓練と評価を行う
- 評価時にミニバッチは使わずテストデータ全体を使って一度に誤差を計算する

同様に精度も求めており、さらにモデルと結果の保存も行う
- **訓練に5分程度必要となるので気長に待つこと**
- 訓練の待ち時間を勉強時間に含めると簡単に文科省の規定時間は満たせるであろう

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
def train(train_loader):
  model.train()
  running_loss = 0
  for i, (images, labels) in enumerate(train_loader): # 一応回数も使えるように
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    loss.backward()
    optimizer.step()
  train_loss = running_loss / len(train_loader)
  return train_loss

def valid(test_loader):
  model.eval() # 今回はDropoutを利用しているので必須
  running_loss = 0
  correct = 0
  total = 0
  with torch.no_grad(): # メモリの効率利用、optimizer.zero_grad()を呼び出す必要がなくなる
    for i, (images, labels) in enumerate(test_loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels) # ロスの計算
      running_loss += loss.item()
      predicted = outputs.max(1, keepdim=True)[1] # 精度の計算
      correct += predicted.eq(labels.view_as(predicted)).sum().item() # 先の例で説明済み
      total += labels.size(0)
  val_loss = running_loss / len(test_loader)
  val_acc = correct / total
  return val_loss, val_acc

loss_list = []
val_loss_list = []
val_acc_list = []
for epoch in range(num_epochs):
  loss = train(train_loader)
  val_loss, val_acc = valid(test_loader)
  print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f'
        % (epoch, loss, val_loss, val_acc))
  # logging
  loss_list.append(loss)
  val_loss_list.append(val_loss)
  val_acc_list.append(val_acc)
print('訓練終了')
# 結果の保存とモデルの保存
np.save('loss_list.npy', np.array(loss_list))
np.save('val_loss_list.npy', np.array(val_loss_list))
np.save('val_acc_list.npy', np.array(val_acc_list))
torch.save(model.state_dict(), 'cnn.pkl')

## 結果表示

アルゴリズム自体は、MNISTと全く同じである

今回もtestデータの方がロスが小さいが、これは前回説明した通り、ミニバッチを用いているため、そのtrainの平均としてのロスと、エポックでの学習が終了した状態でのtestのロスを比較しているためである

また、データ拡張を用いた際も、testデータの方がロスが小さくなる傾向にある

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# plot learning curve
plt.figure()
plt.plot(range(num_epochs), loss_list, 'r-', label='train_loss')
plt.plot(range(num_epochs), val_loss_list, 'b-', label='test_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()

plt.figure()
plt.plot(range(num_epochs), val_acc_list, 'g-', label='val_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.grid()

## 訓練済みモデルを用いた予測

画像を入力しモデルが機能していることを確認する

- 簡単なモデルであるため、精度は期待しないこと
- `shffle=True`としているため、このセルを実行する度にランダムに画像が選ばれ、評価される

In [ ]:
dataiter = iter(torch.utils.data.DataLoader(cifar10, shuffle=True))
images, labels = dataiter.next()  # サンプルを1つだけ取り出す

plt.imshow(np.transpose(images[0], (1, 2, 0)))  # チャンネルを一番後ろに
plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # ラベルとメモリを非表示に
plt.show()

model.eval()  # 評価モード
imagee, labele = images.cuda(), labels.cuda()  # GPU対応
with torch.no_grad():
  labely = model(imagee)
print("正解:", classes[labels[0]],
      "予測結果:", classes[labely.argmax().item()])

# 課題9-1 (CNN)

**(課題1) CIFAR-10のコードを改良して精度を向上させよう**

レポート条件
- 上記の例の結果を添付する
- 改良し、上記よりも少ないエポック数で、上記よりも良い結果を得るモデルを作成して添付する
- 改良は、既習済みの内容であればどこを改良してもよい
  - データは指定したデータ以外使わないこと
  - ヒントはMNISTをちゃんと見るとすぐにわかる(じゃぁ、BNを使えと素直に言えばよいじゃん！ということになるが、皆さんの自由な発想は奪わないようにしたい)
- 評価部分などは一切触らないこと

<font color='red'>他人と同じ内容、レポートになる可能性はゼロになるはずです。この課題については、Turnitinの剽窃チェックの結果も点数に反映されます。</font>

**(課題2) CIFAR-10のコードについてデータ拡張を追加しよう**

レポート条件
- 領域のランダム消去(tansforms.RandomErasing)を**追加で**用いて評価しよう
  - 消去確率は0.5とすること
  - 正しく画像が一部消去されていることも確認すること
- 課題1とは独立です
  - なので、結果がよくなることは期待していません
- その他の部分は修正せずに提出すること

<font color='red'>ランダム消去なので結果は基本的に全員異なるはずです。少なくとも学習過程は違うはずですね。</font>

以下、PyTorchのページにあるマニュアルの簡単な翻訳です
- 課題は利用すればよいので、オプションにこだわる必要はありません

---
```torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)```

画像内の矩形領域をランダムに選択し、そのピクセルを消去します

| | |
|:--|:--|
|p| ランダム消去操作が実行される確率|
|scale|入力画像に対する消去領域の割合の範囲|
|ratio| 消去領域のアスペクト比(縦横比)の範囲|
|value=0| 消去値を指定<br>単一のint数値の場合は全ピクセルを消去<br>3要素のタプルはRGBに対応する<br>'random' の文字列は各ピクセルをランダムな値で消去|
|inplace=False| 参照されたデータのみ矩形消去するか(False)、<br>元データそのものを矩形消去するか(True)を選択|

サンプルコード
```
transform = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
  transforms.RandomErasing(),
  ])
```
---

**課題1と課題2の二つのノートブックは、一つのノートブックに記載すること**

その他の提出方法はこれまでと同様とする

# CNNによるStyle Transfer
## 考え方
写真をピカソやゴッホのようなスタイルに変換できるスマホアプリ(Prisma)が話題となったことがあるが、このような技術をStyle Transferと呼ぶ
- 特にDNNを用いる高品質なStyle Transfer手法が2016年に出現、その後様々な改良がなされている

ここでは、その基本となるGatysらの研究に基づき実装を行う

まず、CNNを用いたVGG19を利用している
- VGG19は画像分類(物体認識)モデルであり、2014年のILSVRCコンペで優勝したモデル
- CNNを用いたSytle Transferは、CNNによる画像分類の仕組みをうまく利用している

VGGなどCNNを用いる画像分類モデルでは、最初の層ほど細かな特徴を扱い、層を進むに従い、分類上重要となる情報が残り、その中で関係のない情報が欠落するような変換が行われる
- つまり、コンテンツとスタイルを分離できるのではないか？というのがこの手法の出発点

この性質を利用して、コンテンツ(例えば街並みの写真、ここでは$\boldsymbol{p}$)を保ったまま、スタイル(たとえばゴッホの絵など、ここでは$\boldsymbol{a}$)を別の画像で獲得したスタイルと入れ替えることを考える

スタイル変換にはVGGから全結合層を除いた部分の重みを利用する

次の図はCNNの各層が画像をどのように表現しているかを示しており、下の街並みの写真にあるように、上位の層で入力画像を復元すると、元の画像と変化は少ないが、より深い層では、
入力画像の詳細な情報が欠落していることがわかる

<img src='http://class.west.sd.keio.ac.jp/dataai/text/stcnn.png' width=600>




## 仕組み
### 損失関数
コンテンツを保ちつつ、スタイルを他の画像のスタイルに近づけるため、**コンテンツの損失+スタイルの損失**を損失関数とし、これを最小化する

#### コンテンツの損失
コンテンツの損失を抑えるため、コンテンツ画像のコンテンツ保持層の重みと、生成画像のコンテンツ重みを比較し、これらが等しくなるようにする
- 生成画像を$\boldsymbol{x}$とする
- $F$および$P$の添え字が意味するのは複数の層$l$の$i$番目チャネルの位置$j$における活性度を表しており、隠れCNN層の出力値に相当する
- つまり、画像であり、深い層において同じ画像出力が得られるようにしている
  - 細かい画像の情報は、スタイル変換で失われても、大筋で元の画像を残していればよい、という発想

$$
\mathcal{L}_{content}(\boldsymbol{p}, \boldsymbol{x}, l)=\frac{1}{2}\sum_{i,j}(F^l_{ij}-P^l_{ij})^2
$$

ここでは、コンテンツ重みは、conv4_2層を利用している
- コードでは、21番目の層のみ考慮するため、$l$は1つでよい

#### スタイルの損失
次に、スタイルの損失は、特徴マップの相関とする

まず、グラム行列$G^l$を次のように定義する
$$
G^l_{ij}=\sum_k(F^l_{jk}-P^l_{jk})^2
$$

スタイルを抜き出す画像のグラム行列$G$と、生成画像のグラム行列$A$を用いて次の値を求める
- 要するにこれらのグラム行列の違いを表現している
$$
E_l=\frac{1}{4N^2_lM^2_l}\sum_{i,j}(G^l_{ij}-A^l_{ij})^2
$$
コンテンツの損失同様、複数の層を考慮するため、最終的に次の式をスタイルの損失とする
$$
\mathcal{L}_{style}(\boldsymbol{a},\boldsymbol{x})=\sum_l^L\boldsymbol{w}_lE_l
$$
ここで、$\boldsymbol{a}$はスタイル画像を、$\boldsymbol{w}_l$は第$l$層の重みを表し、ここでは、conv1_1, conv2_2, conv3_1, conv4_1, conv5_1層を用いる
- つまり、スタイル画像が持つCNNの重みにできるだけ近づけることで、そのスタイルを残そう、という発想

#### 全体の損失

$\alpha$と$\beta$をそれぞれの損失重み係数とすると、
$$
mathcal{L}_{content}(\boldsymbol{p}, \boldsymbol{a}, \boldsymbol{x})=
\alpha\mathcal{L}_{content}(\boldsymbol{p}, \boldsymbol{x})+\beta\mathcal{L}_{style}(\boldsymbol{a},\boldsymbol{x})
$$
が求める損失となる

<img src='http://class.west.sd.keio.ac.jp/dataai/text/stloss.png' width=600>

## その他

- VGGにおいて、MaxPoolingが用いられているが、AveragePoolingの方が画像変換には適しており、良い結果が得られることが知られている
  - ここでは学習済みモデルを用いているため、MaxPoolingである
- conv5_2を用いる方が良いという実装あり
  - とくに修正していない
- 変換過程がわかるように、動画を生成させている
  - 生成例は次の通りで、直接クリックしても再生できない場合は、ファイルを保存して再生するとよい
  - [こちら夜の都会バージョン](https://keio.box.com/shared/static/bhxd114ih0g4lvfycziryve573os2514)と[こちらゴッホ+凛尾君バージョン](https://keio.box.com/shared/static/jgnobefkgc9tbdox08tpk95dmwig9lk2)、[ラッセン+凛尾君バージョン](https://keio.box.com/shared/static/p7fqj84ha1haqannzbnlqzjq4qeelbfi)、[ゲルニカ+凛尾君バージョン](https://keio.box.com/shared/static/saeczpp7w0mcg6wg6b8nbdz4xjvvcewi)

- やはりうまく生成させるためには、画像ごとにパラメタのチューニングが必要
  - 触りやすいのは、$\alpha$と$\beta$であろう
  - スタイルの適用具合の調整が行いやすい
生成例は次の通り

論文から

<img src='http://class.west.sd.keio.ac.jp/dataai/text/start.png' width=600>

お試し例

<img src='http://class.west.sd.keio.ac.jp/dataai/text/riost.png' width=600>


## 実行コード

Pillowを入れるが、Google Colaboratoryの場合不要

In [ ]:
!pip install Pillow

いつもの初期化コード

In [ ]:
import torch
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
device = torch.device(cuda if torch.cuda.is_available() else "cpu")

今回はVGGと呼ばれる著名な学習済み画像認識モデルを利用する
- PyTochはこのような著名な画像認識モデルを含む様々なモデルをはじめから準備している
- 学習済みモデルのパラメータを変更しないように、AutogradをOFFにしておく

In [ ]:
vgg = models.vgg19(pretrained=True).features
vgg.to(device)
for param in vgg.parameters():
  param.requires_grad_(False)

画像を読み込むための関数の定義
- 画像のサイズを整え、正規化する
- PillowのImage.openは、BMP, EPS、GIF、ICO、JPEG、PNG、PPM、TIFF、WebP、XBMなどなど、様々なフォーマットに対応している
- なので、皆さんは結構どのような画像を持ってきてもうまく動くはず
- `transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)`は説明済

In [ ]:
def load_image(img_path, max_size=400, shape=None):
  image = Image.open(img_path).convert('RGB')
  if max(image.size) > max_size:
    size = max_size
  else:
    size = max(image.size)
  if shape is not None:
    size = shape
  in_transform = transforms.Compose([
               transforms.Resize(size),
               transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), 
                                    (0.5, 0.5, 0.5))])
  image = in_transform(image).unsqueeze(0)   
  return image

以下に用いる画像をロードしているが、好きな画像を利用するにはファイルを上書きすればよい
- 左のcontentフォルダにデータが入るが、好きな画像に入れ替えてみよう
- jpgじゃなくてもメジャーなフォーマットであれば大丈夫

In [ ]:
import os
if not os.path.exists('contents.jpg'):
  #!wget "https://drive.google.com/uc?export=download&id=1OhdW3HJwVRUOwGTOMxhj2YP8bDwWjMkk" -O contents.jpg
  !wget https://keio.box.com/shared/static/nla0l244kq5o30p5eh9iop0sk5jsl6ap -O contents.jpg

if not os.path.exists('style.jpg'):
  #!wget "https://drive.google.com/uc?export=download&id=1OhdW3HJwVRUOwGTOMxhj2YP8bDwWjMkk" -O style.jpg
  !wget https://keio.box.com/shared/static/az9yr07d5j5b3agtj7i8v5iwbso7za8u -O style.jpg

左のフォルダにドラッグして自分の画像ファイルを入れて、次のファイル名を変更するとよい
- スタイルは基本変更しなくてもよいが、「画風が独特の絵画」などを入れると楽しめるであろう

In [ ]:
content = load_image('contents.jpg').to(device)
style = load_image('style.jpg').to(device)

途中経過などで利用するため、**画像をメモリからコピーして取り出し**、正規化しているのでを元に戻してきちんとした画像にする関数を定義

- np.arrayからPillow(PIL)への変換に、transpose(1,2,0)を行う
  - これは並び順の変換で(color, x, y)という順番であるため、(x, y, color)にする
- 平均0、標準偏差0.5の正規表現($-1<pixels<1$)で正規化されたデータを0から1の値にするため0.5倍して0.5を足す
- image.clipで値の範囲を0から1に限定する

In [ ]:
def im_convert(tensor):
  image = tensor.to("cpu").clone().detach()
  image = image.numpy().squeeze() # 大きさが1の次元を纏めて削除
  image = image.transpose(1,2,0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

読み込んだ2つの画像を実際に表示してみる

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(im_convert(content))
ax1.axis("off")
ax2.imshow(im_convert(style))
ax2.axis("off")

既に説明した通り、必要な層のパラメータを取得する関数

In [ ]:
def get_features(image, model): 
  layers = {'0': 'conv1_1',
            '5': 'conv2_1', 
            '10': 'conv3_1', 
            '19': 'conv4_1',
            '21': 'conv4_2',  # Content Extraction
            '28': 'conv5_1'}     
  features = {}
  for name, layer in model._modules.items():
    image = layer(image)
    if name in layers:
      features[layers[name]] = image        
  return features

コンテンツとスタイルそれぞれの必要な層のパラメータを取得

In [ ]:
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)

グラム行列($\boldsymbol{A} \cdot \boldsymbol{A})$を求める
- 既に説明した通り損失を得るために必要

In [ ]:
def gram_matrix(tensor):
  _, d, h, w = tensor.size()
  tensor = tensor.view(d, h * w)
  gram = torch.mm(tensor, tensor.t())
  return gram

グラム行列の実際の計算

In [ ]:
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

各層の特徴をどの程度利用するかの比率表
- ここは各自で自由に修正しても面白いであろう

In [ ]:
style_weights = {'conv1_1': 1.,
                 'conv2_1': 0.75,
                 'conv3_1': 0.2,
                 'conv4_1': 0.2,
                 'conv5_1': 0.2}
content_weight = 1  # alpha
style_weight = 1e6  # beta 1e6がデフォルトだが1e4などでも破綻はしない

データを修正するが、オリジナルを取っておきたいのでクローニングする
- 違うメモリに画像データを格納する
- これをそのままNNにかけるのでAutogradをONにする

In [ ]:
target = content.clone().requires_grad_(True).to(device)

各種ハイパーパラメタ
- 特に修正する必要はないと思が、STEPはどの程度画像を触るかに影響するので、各自で調整するとよいであろう

In [ ]:
show_every = 300
optimizer = optim.Adam([target], lr=0.003)
steps = 2100
height, width, channels = im_convert(target).shape
image_array = np.empty(shape=(300, height, width, channels))
capture_frame = steps/300
counter = 0

実際の処理
- 既に説明した通りにロスを計算している
- ロスを自由に設計してもPyTorchは、.backwordで一発逆誤差伝播法の演算が可能であることがわかる
- 上記capture_frameで指定された変換動画を作成するための画像データ取得間隔毎に、画像をimage_arrayに蓄えている

In [ ]:
for ii in range(1, steps+1):
  target_features = get_features(target, vgg) # VGGから特徴量を抜き出す
  content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2) # 損失関数
  style_loss = 0
  for layer in style_weights: # スタイルの抽出
    target_feature = target_features[layer]
    target_gram = gram_matrix(target_feature)
    style_gram = style_grams[layer]
    layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
    _, d, h, w = target_feature.shape
    style_loss += layer_style_loss / (d * h * w)
  total_loss = content_weight * content_loss + style_weight * style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()
  if  ii % show_every == 0:
    print('Total loss: ', total_loss.item())
    print('Iteration: ', ii)
    plt.imshow(im_convert(target))
    plt.axis("off")
    plt.show()
  if ii % capture_frame == 0:
    image_array[counter] = im_convert(target)
    counter = counter + 1

最終結果の表示

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
ax1.imshow(im_convert(content))
ax1.axis('off')
ax2.imshow(im_convert(style))
ax2.axis('off')
ax3.imshow(im_convert(target))
ax3.axis('off')

途中結果(image_array)を用いて変換動画(output.mp4)を作成

In [ ]:
import cv2 
frame_height, frame_width, _ = im_convert(target).shape
vid = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))
for i in range(0, 300):
  img = image_array[i]
  img = img*255
  img = np.array(img, dtype = np.uint8)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  vid.write(img)
vid.release()

結果をブラウザからダウンロードさせる
- こんなこともできるよ、ということ
  - Google Colaboratoryで実行しているときのみ、ファイルをダウンロードできるようにしている

In [ ]:
import sys
moduleList = sys.modules
if 'google.colab' in moduleList:
  from google.colab import files
  files.download('output.mp4')

# 課題9-2 (StyleTransfer)

なんでもよいので、自分の好きな画像とスタイル画像を用いて、変換してみよう！

## レポート条件

レポートにコンテンツ画像、スタイル画像、変換後画像をそれぞれ添付しなさい
  - スタイル画像は上記のままでも問題ありません

<font color='red'>こちらも、もちろん皆さん違う写真であることを期待しています。</font>